# TP - Agents Conversationnels

### Authors

Théo DAUDIN, Noémie HENRY, Paul BERTIN, Julien PINCON



## Instructions partie pratique

 - Travail en groupe de 3 à 4 étudiant.e.s
 - **Objectif:** Réalisation rapide d’un  chatbot
 - **Modèle:** Seq2seq https://github.com/tensorlayer/seq2seq-chatbot
 - Source de données variable (Cornell, twitter, Gutenberg https://github.com/ricsinaruto/gutenberg-dialog)
 - Rapport de 4 pages maximum:
	- explication du modèle ou des modèles utilisés (architecture)
	- variation des (hyper)-paramètres
	- données utilisées
	- résultats obtenus
	- conclusion et perspectives d’utilisation et/ou de recherches

## [Accéder au rapport](https://docs.google.com/document/d/11O_3wc9vJYEHj803iSQmO-h3pAqsCjCVNykn6Yrkxlc/edit?usp=sharing )


In [ ]:
!pip install tensorlayer

     |████████████████████████████████| 363 kB 31.2 MB/s 
     |████████████████████████████████| 3.3 MB 10.4 MB/s 
     |████████████████████████████████| 13.3 MB 22.4 MB/s 
     |████████████████████████████████| 178 kB 23.1 MB/s 
     |████████████████████████████████| 6.1 MB 52.9 MB/s 
     |████████████████████████████████| 1.9 MB 54.4 MB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import tensorflow as tf
import tensorlayer as tl

print("Tensorflow version : ",tf.__version__)
print("Tensorlayer version : ",tl.__version__)

Tensorflow version :  2.4.1
Tensorlayer version :  2.2.3


## Inspiration (vol) d'un chatbot Twitter (https://github.com/suriyadeepan/practical_seq2seq/blob/master/03-Twitter-chatbot.ipynb)

In [ ]:
!pip install tensorflow==2.7.0
!pip install tensorflow-addons[tensorflow]
!pip install datasets.twitter

     |████████████████████████████████| 489.6 MB 16 kB/s 
     |████████████████████████████████| 463 kB 42.1 MB/s 
     |████████████████████████████████| 13.3 MB 14.3 MB/s 
     |████████████████████████████████| 2.1 MB 11.0 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.4.0
    Not uninstalling tensorflow-estimator at /shared-libs/python3.7/py/lib/python3.7/site-packages, outside environment /root/venv
    Can't uninstall 'tensorflow-estimator'. No files were found to uninstall.
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.4.1
    Not uninstalling tensorflow at /shared-libs/python3.7/py/lib/python3.7/site-packages, outside environment /root/venv
    Can't uninstall 'tensorflow'. No files were found to uninstall.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 1.1 MB 11.1 MB/s 
You should consider u

In [ ]:
import tensorflow as tf
import numpy as np

# preprocessed data
# /work/datasets/twitter/data.py
from work.datasets.twitter import data
from work import data_utils

In [ ]:
# load data from pickle and npy files
metadata, idx_q, idx_a = data.load_data(PATH='datasets/twitter/')
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_q, idx_a)

In [ ]:
# parameters 
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 16
xvocab_size = len(metadata['idx2w'])  
yvocab_size = xvocab_size
emb_dim = 1024

In [ ]:
import seq2seq_wrapper

/root/venv/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.5.0 and strictly below 2.8.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [ ]:
import importlib
importlib.reload(seq2seq_wrapper)

<module 'seq2seq_wrapper' from '/work/seq2seq_wrapper.py'>

In [ ]:
model = seq2seq_wrapper.Seq2Seq(xseq_len=xseq_len,
                               yseq_len=yseq_len,
                               xvocab_size=xvocab_size,
                               yvocab_size=yvocab_size,
                               ckpt_path='ckpt/twitter/',
                               emb_dim=emb_dim,
                               num_layers=3
                               )

<log> Building Graph 

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
val_batch_gen = data_utils.rand_batch_gen(validX, validY, 256)
test_batch_gen = data_utils.rand_batch_gen(testX, testY, 256)
train_batch_gen = data_utils.rand_batch_gen(trainX, trainY, batch_size)

In [ ]:
sess = model.train(train_batch_gen, val_batch_gen)

In [ ]:
sess = model.restore_last_session()

In [ ]:
input_ = test_batch_gen.__next__()[0]
output = model.predict(sess, input_)
print(output.shape)

In [ ]:
replies = []
for ii, oi in zip(input_.T, output):
    q = data_utils.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded = data_utils.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split(' ')
    if decoded.count('unk') == 0:
        if decoded not in replies:
            print('q : [{0}]; a : [{1}]'.format(q, ' '.join(decoded)))
            replies.append(decoded)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6a5bfce1-7e75-414c-b298-2f108ab8125e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>